# Testing swiftly API
Links:
* https://realtime-docs.goswift.ly/intro
* http://gtfsbook.com/
* https://geoffboeing.com/2019/09/urban-street-network-orientation/#more-3873

Run this Notebook on a SciServer container with the Geo image
This notebook needs some libraries to be installed.

TBD describe which

In [ ]:
import requests
import json
from google.transit import gtfs_realtime_pb2
from datetime import datetime
import pytz
import time

In [ ]:
from ipyleaflet import Map, Marker, CircleMarker, Polygon, Rectangle
from ipywidgets import HTML
import numpy as np

In [ ]:
API_key='e6c3156850ece799033be8a177b43095'
#API Documentation: https://realtime-docs.goswift.ly/
agencyKey='mta-maryland'
#agencyKey='mta-newyork'


In [ ]:
base_url='https://api.goswift.ly/'
headers={"Authorization":API_key,'Accept':'application/json'}

In [ ]:
r=requests.get(url=base_url+'test-key',headers=headers)
test=json.loads(r.content.decode())
test

In [ ]:
r=requests.get(url=base_url+'info/'+agencyKey,headers=headers)
info=json.loads(r.content.decode())
info

# plot area on map

In [ ]:
def addVehicle2Map(m,vehicle):
    '''
    this function defines markers and puts these on a ipyleaflet Map.
    Clicking on them gives a pop-up with links to images/drawings.
    '''
    loc=(vehicle.position.latitude,vehicle.position.longitude)
    ts=vehicle.timestamp
    ts=datetime.fromtimestamp(entity.vehicle.timestamp)
    utc_now = pytz.utc.localize(ts)
    pst_now = utc_now.astimezone(pytz.timezone("America/New_York"))
    marker = CircleMarker()
    marker.location = loc
    marker.radius = 1
    marker.color = "red"
    marker.fill_color = "red"

    message=HTML()
    message.value="Vehicle: {0} [{1}]".format(vehicle.vehicle.id,pst_now)
    
    marker.popup=message
    return marker


In [ ]:
ll = (info['data']['extent']['minLat'],info['data']['extent']['minLon'])
ur = (info['data']['extent']['maxLat'],info['data']['extent']['maxLon'])

center=tuple((np.array(ll)+np.array(ur))/2)

In [ ]:
m = Map(center=center, zoom=10)
m.scroll_wheel_zoom=True
m.layout.height='700px'
m.layout.width='700px'

rectangle = Rectangle(bounds=(ll,ur),weight=2,fill=False)

m.add_layer(rectangle)

display(m)

In [ ]:
while True:
    r=requests.get(url=base_url+'real-time/'+agencyKey+"/gtfs-rt-vehicle-positions",headers=headers)

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(r.content)
    feed.entity
    markers=[]
    for ix,entity in enumerate(feed.entity):
        if entity.HasField('vehicle'):
            markers.append(addVehicle2Map(m,entity.vehicle))
            if ix > 10:
                break
    marker_cluster = MarkerCluster(markers=markers)
    m.add_layer(marker_cluster)
    time.sleep(30)
    

In [ ]:
r=requests.get(url=base_url+'info/'+agencyKey+"/routes",headers=headers)
routes=json.loads(r.content.decode())
routes

In [ ]:
params={'route':'11735,11744','verbose':False}
r=requests.get(url=base_url+'info/'+agencyKey+"/routes",headers=headers,params=params)
route=json.loads(r.content.decode())
route

In [ ]:
params={'route':'11735','verbose':False}
r=requests.get(url=base_url+'real-time/'+agencyKey+"/vehicles",headers=headers,params=params)
route=json.loads(r.content.decode())
for vehicle in route['data']['vehicles']:
    print(datetime.fromtimestamp(vehicle['loc']['time']))

In [ ]:
vehicle